In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf

In [2]:
interpreter = tf.lite.Interpreter(model_path="./alt_cnn_model_uint8.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)


Input details: [{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([ 1, 96, 96,  1]), 'shape_signature': array([-1, 96, 96,  1]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003921568859368563, -128), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 16, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [5]:
from PIL import Image
import numpy as np
import tensorflow as tf

# Load and preprocess image
currentImage = Image.open("EspIdfCode/frame.png")
processedImage = currentImage.resize([96, 96], Image.Resampling.LANCZOS).convert("L")

img_np = np.array(processedImage, dtype=np.float32) / 255.0


# Add batch and channel dimensions
img_scaled = img_np[np.newaxis, ..., np.newaxis]  # (1, 96, 96, 1)

# Get quantization parameters
input_details = interpreter.get_input_details()
input_scale, input_zero_point = input_details[0]['quantization']

# Quantize
img_quant = (img_scaled / input_scale + input_zero_point).round().astype(np.int8)

# Run inference
interpreter.set_tensor(input_details[0]['index'], img_quant)
interpreter.invoke()

# Get output
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])
output_scale, output_zero_point = output_details[0]['quantization']
prob_class0 = (output_data[0][0] - output_zero_point) * output_scale
prob_class1 = 1 - prob_class0

print("Class 0 probability:", prob_class0)
print("Class 1 probability:", prob_class1)


Class 0 probability: 0.0859375
Class 1 probability: 0.9140625


In [6]:
print("Min/max:", img_scaled.min(), img_scaled.max())


Min/max: 0.019607844 0.9137255
